In [3]:
!pip install pyarrow



     --------------------------------------- 19.5/19.5 MB 19.3 MB/s eta 0:00:00


In [1]:
import pyarrow as pa
import pandas as pd
import numpy as np
import json
import pyarrow.parquet as pq

In [2]:
df = pd.DataFrame(
  { 'temp': [12.1, 11, 13, 10, 10],
    'rain': [9.2, 10.0, 2.2, 0.2, 0.4] },
    index=pd.DatetimeIndex(['2020-10-12',
                            '2020-10-13',
                            '2020-10-14',
                            '2020-10-15',
                            '2020-10-16'],
                           name='date')
)

In [3]:
a=dict({'a':[1,2,3],'b':[2,3,4]})
a=pd.DataFrame(a)
a.to_dict('index')
a

,a,b
0,1,2
1,2,3
2,3,4


In [40]:
custom_meta_content = {
    'user': 'Wáng Fān',
    'coord': '55.9533° N, 3.1883° W',
    'time': '2020-10-17T03:59:59+0000'  # ISO-8601
}
c=pd.DataFrame(custom_meta_content,index=["user","coord","time"])

In [141]:
custom_meta_key = 'weatherapp.iot'

In [86]:
table = pa.Table.from_pandas(df)

In [87]:
print(table.schema.metadata)

{b'pandas': b'{"index_columns": ["date"], "column_indexes": [{"name": null, "field_name": null, "pandas_type": "unicode", "numpy_type": "object", "metadata": {"encoding": "UTF-8"}}], "columns": [{"name": "temp", "field_name": "temp", "pandas_type": "float64", "numpy_type": "float64", "metadata": null}, {"name": "rain", "field_name": "rain", "pandas_type": "float64", "numpy_type": "float64", "metadata": null}, {"name": "date", "field_name": "date", "pandas_type": "datetime", "numpy_type": "datetime64[ns]", "metadata": null}], "creator": {"library": "pyarrow", "version": "9.0.0"}, "pandas_version": "1.3.4"}'}


In [133]:
custom_meta_content2 = {
    'user': np.array([1,2,3,4,5]).tolist(),
    'coord': '55.9533° N, 3.1883° W',
    'time': '200-10-17T03:59:59+0000'  # ISO-8601
}
custom_meta_json = json.dumps(custom_meta_content2)
custom_meta_content2

{'user': [1, 2, 3, 4, 5],
 'coord': '55.9533° N, 3.1883° W',
 'time': '200-10-17T03:59:59+0000'}

In [88]:
custom_meta_json = json.dumps(custom_meta_content2)
existing_meta = table.schema.metadata
combined_meta = {
    custom_meta_key.encode() : df,
    **existing_meta
}

In [89]:
table = table.replace_schema_metadata(combined_meta)

TypeError: expected bytes, DataFrame found

In [70]:
print(table.schema.metadata)

{b'weatherapp.iot': b'{"user": "W\\u00e1ng F\\u0101n", "coord": "55.9533\\u00b0 N, 3.1883\\u00b0 W", "time": "2020-10-17T03:59:59+0000"}', b'pandas': b'{"index_columns": ["date"], "column_indexes": [{"name": null, "field_name": null, "pandas_type": "unicode", "numpy_type": "object", "metadata": {"encoding": "UTF-8"}}], "columns": [{"name": "temp", "field_name": "temp", "pandas_type": "float64", "numpy_type": "float64", "metadata": null}, {"name": "rain", "field_name": "rain", "pandas_type": "float64", "numpy_type": "float64", "metadata": null}, {"name": "date", "field_name": "date", "pandas_type": "datetime", "numpy_type": "datetime64[ns]", "metadata": null}], "creator": {"library": "pyarrow", "version": "9.0.0"}, "pandas_version": "1.3.4"}'}


In [71]:
pq.write_table(table, 'example.parquet', compression='GZIP')

# read parquet

In [4]:
restored_table = pq.read_table('spectrum.parquet')

In [5]:
restored_table

pyarrow.Table
E: double
TOC013_total_mu: double
HCN010_total_mu: double
01_001_total_mu: double
HCN028_total_mu: double
02-005_total_mu: double
TOC140_total_mu: double
TOC038_total_mu: double
03_005_total_mu: double
22_021_total_mu: double
HCN003_total_mu: double
HCN015_total_mu: double
HCN040_total_mu: double
TOC079_total_mu: double
COC013_total_mu: double
HCN018_total_mu: double
03-006_total_mu: double
HCN006_total_mu: double
CUBE00_total_mu: double
HCO010_total_mu: double
01-002_total_mu: double
TOC225_total_mu: double
----
E: [[11559.2482487658,11559.3482487658,11559.4482487658,11559.548248765801,11559.648248765801,...,11621.748248766027,11621.848248766028,11621.948248766028,11622.048248766028,11622.148248766029]]
TOC013_total_mu: [[0.24995968168727997,0.25501505454044665,0.26027082417783226,0.2657364314092692,0.27142131704459066,...,1.006728817475476,1.0061685372628868,1.0056024298628228,1.0050308492747662,1.004454149498199]]
HCN010_total_mu: [[0.23467531207289669,0.23886442230850

In [156]:
restored_table=restored_table.append_column('E2',pa.array(np.arange(0,a)))
print(restored_table)
custom_meta_json = json.dumps(custom_meta_content2)
existing_meta = restored_table.schema.metadata
combined_meta = {
    custom_meta_key.encode() : custom_meta_json,
    **existing_meta
}
table = restored_table.replace_schema_metadata(combined_meta)

table

pyarrow.Table
E: double
05_007_total_mu: double
E2: int32
----
E: [[11559.0985867686,11559.1985867686,11559.2985867686,11559.3985867686,11559.498586768601,...,11621.498586768826,11621.598586768827,11621.698586768827,11621.798586768828,11621.898586768828]]
05_007_total_mu: [[0.24257464755470343,0.24721222729395864,0.2520269700027552,0.25702818961285184,0.262225200056007,...,1.0076905877942073,1.0071802307625788,1.0066644036616468,1.0061434355110233,1.0056176553303207]]
E2: [[0,1,2,3,4,...,624,625,626,627,628]]


pyarrow.Table
E: double
05_007_total_mu: double
E2: int32
----
E: [[11559.0985867686,11559.1985867686,11559.2985867686,11559.3985867686,11559.498586768601,...,11621.498586768826,11621.598586768827,11621.698586768827,11621.798586768828,11621.898586768828]]
05_007_total_mu: [[0.24257464755470343,0.24721222729395864,0.2520269700027552,0.25702818961285184,0.262225200056007,...,1.0076905877942073,1.0071802307625788,1.0066644036616468,1.0061434355110233,1.0056176553303207]]
E2: [[0,1,2,3,4,...,624,625,626,627,628]]

In [157]:
pq.write_table(table, 'example.parquet', compression='GZIP')

In [158]:
restored_table = pq.read_table('example.parquet')


In [159]:
restored_table

pyarrow.Table
E: double
05_007_total_mu: double
E2: int32
----
E: [[11559.0985867686,11559.1985867686,11559.2985867686,11559.3985867686,11559.498586768601,...,11621.498586768826,11621.598586768827,11621.698586768827,11621.798586768828,11621.898586768828]]
05_007_total_mu: [[0.24257464755470343,0.24721222729395864,0.2520269700027552,0.25702818961285184,0.262225200056007,...,1.0076905877942073,1.0071802307625788,1.0066644036616468,1.0061434355110233,1.0056176553303207]]
E2: [[0,1,2,3,4,...,624,625,626,627,628]]

In [6]:
h=list(restored_table.schema.metadata.keys())
print(h)
js=restored_table.schema.metadata[h[1]]
restored_meta = json.loads(js)

[b'TOC225', b'01-002', b'HCO010', b'CUBE00', b'HCN006', b'03-006', b'HCN018', b'COC013', b'TOC079', b'HCN040', b'HCN015', b'HCN003', b'22_021', b'03_005', b'TOC038', b'TOC140', b'02-005', b'HCN028', b'01_001', b'HCN010', b'TOC013', b'pandas']


In [188]:
restored_meta.keys()

dict_keys(['E', 'site_1_mu', 'site_2_mu', 'site_3_mu', 'site_5_mu', 'site_8_mu', 'site_9_mu', 'site_11_mu', 'site_20_mu', 'total_mu'])

In [15]:
a=np.array([1,2,3,4,5,6])
np.savetxt('a.ini',a,fmt='%.8f')
np.loadtxt('a.ini')

array([1., 2., 3., 4., 5., 6.])